In [ ]:
"""
Created on Mon Jan 20 11:50 2025

Compare the file currently used for IPSL-CM7 and the one I created to make them look similar

@author: Clara Burgard
"""

In [1]:
import xarray as xr
from multimelt.constants import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ DATA

In [3]:
inputpath_raw='/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/raw/'
inputpath_interim='/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/interim/'
inputpath_raw2 = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'


In [4]:
file_IPSL = xr.open_dataset(inputpath_raw + 'eORCA1.4.2_runoff-icb_DaiTrenberth_Depoorter.nc')
file_Davison = xr.open_dataset(inputpath_interim + 'Davison23_basal_melt_pattern_on_eORCA1.nc')
file_current = xr.open_dataset(inputpath_interim + 'masks_for_eORCA1_prescribedmeltinopencav.nc')

In [44]:
domain_cfg = xr.open_dataset(inputpath_raw2 + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc')

In [6]:
cell_area = domain_cfg['e1t'] * domain_cfg['e2t']

In [7]:
melt_converted = file_current['melt_isf'] / yearinsec *10**(12) / cell_area 

In [ ]:
melt_IPSL_Gt = file_IPSL['sornfisf'].isel(time_counter=0) * yearinsec * 10**(-12) * cell_area

In [ ]:
melt_IPSL_Gt.plot()

In [ ]:
melt_IPSL_Gt.where(melt_IPSL_Gt != 0).plot(vmin=-5,vmax=5,cmap = 'coolwarm')

In [ ]:
file_current['melt_isf'].sum()

In [ ]:
plt.figure()
(file_current['melt_isf']).plot(vmin=-5,vmax=5,cmap = 'coolwarm')

In [ ]:
(melt_converted - file_IPSL['sornfisf'].isel(time_counter=0))/file_IPSL['sornfisf'].isel(time_counter=0)).plot(vmax=1)

In [ ]:
(file_current['melt_isf'] - melt_IPSL_Gt).plot()

In [ ]:
melt_converted.plot()

In [ ]:
modified_file_IPSL = file_IPSL.copy()

In [ ]:
modified_file_IPSL['sornfisf'] = file_IPSL['sornfisf'] * 0 + melt_converted.where(np.isfinite(melt_converted),0).squeeze()
modified_file_IPSL['sornfisf'].attrs['long_name'] = 'runoff from antarctica iceshelves (Davison 2023)'

In [ ]:
modified_file_IPSL['sodepmax_isf'] = file_IPSL['sodepmax_isf'] * 0 + file_current['z_max'].where(np.isfinite(file_current['z_max']),0).squeeze()
modified_file_IPSL['sodepmin_isf'] = file_IPSL['sodepmin_isf'] * 0 + file_current['z_min'].where(np.isfinite(file_current['z_min']),0).squeeze()

In [ ]:
modified_file_IPSL.attrs['description2'] = 'south of 60 south : Davison Freshwater Discharge from basal front to grounding line iceshelves melting'
modified_file_IPSL.attrs['reference3'] = 'B.J. Davison, A.E. Hogg, N. Gourmelen, L. Jakob, J. Wuite, T. Nagler, C.A. Greene, J. Andreasen, M.E. Engdahl, 2023: Annual mass budget of Antarctic ice shelves from 1997 to 2021. Science Advances, doi:10.1126/sciadv.adi0186'
modified_file_IPSL.attrs['creation'] = 'Mon Jan 20 16:11 2025'

In [ ]:
modified_file_IPSL.attrs['history5'] = 'modified from eORCA1.4.2_runoff-icb_DaiTrenberth_Depoorter.nc by replacing the ice shelf freshwater flux by values from Davison and prescribing the flux in large cavities to open them in NEMO (contact Clara Burgard)' 

In [ ]:
file_IPSL['sornfisf']

In [ ]:
modified_file_IPSL.to_netcdf(inputpath_interim + 'eORCA1.4.2_runoff-icb_DaiTrenberth_Davison.nc')

In [ ]:
(modified_file_IPSL['sodepmax_isf'] - file_IPSL['sodepmax_isf']).isel(time_counter=0).plot()

In [ ]:
modified_file_IPSL['sodepmax_isf'].plot()

CHECKS

In [52]:
file_IPSL = xr.open_dataset(inputpath_raw + 'eORCA1.4.2_runoff-icb_DaiTrenberth_Depoorter.nc')
modified_file_IPSL = xr.open_dataset(inputpath_interim + 'eORCA1.4.2_runoff-icb_DaiTrenberth_Davison.nc')

ALSO PREPARE THE DOMAIN CFG BY CLOSING MOST CAVITIES

In [45]:
new_domain_cfg = domain_cfg.copy()
for vvar in ['bottom_level','top_level','isf_draft','bathy_metry','mask_opensea','mask_csgrpglo']:
    print(vvar)
    new_domain_cfg[vvar] = new_domain_cfg[vvar].where(np.isnan(file_current['mask_isf_closed']), 0)

bottom_level
top_level
isf_draft
bathy_metry
mask_opensea
mask_csgrpglo


In [49]:
new_domain_cfg.to_netcdf(inputpath_interim + 'eORCA1.4.3_OpenSeas_Open6largestISF_ModStraights_domain_cfg.nc')

CHECK AT WHICH DEPTH FWF IS INJECTED

In [78]:
file_current['mask_isf_open'].plot()

In [79]:
zmin_opencav = new_domain_cfg['nav_lev'].sel(z=new_domain_cfg['top_level']).where(file_current['mask_isf_open'])
zmax_opencav = zmin_opencav.where(np.isnan(file_current['mask_isf_open']), zmin_opencav + 5)

In [95]:
new_modified_file_IPSL = modified_file_IPSL.copy()
new_modified_file_IPSL['sodepmin_isf'] = new_modified_file_IPSL['sodepmin_isf'].where(np.isnan(file_current['mask_isf_open']), zmin_opencav).squeeze()
new_modified_file_IPSL['sodepmax_isf'] = new_modified_file_IPSL['sodepmax_isf'].where(np.isnan(file_current['mask_isf_open']), zmax_opencav).squeeze()

In [97]:
new_modified_file_IPSL.to_netcdf(inputpath_interim + 'eORCA1.4.2_runoff-icb_DaiTrenberth_Davison_open6largestISF.nc', unlimited_dims=['time_counter'])

In [101]:
(new_modified_file_IPSL['sodepmax_isf'].isel(time_counter=0) - new_modified_file_IPSL['sodepmin_isf'].isel(time_counter=0)).plot(vmax=10)

In [104]:
new_domain_cfg['nav_lev'].shift(z=1) - new_domain_cfg['nav_lev']

<xarray.DataArray 'nav_lev' (z: 75)>
array([         nan,   -1.0500953,   -1.1118264,   -1.1885982,
         -1.2840815,   -1.4026723,   -1.5494852,   -1.7302313,
         -1.9509296,   -2.2173586,   -2.5342836,   -2.90448  ,
         -3.327814 ,   -3.800684 ,   -4.3162613,   -4.8656425,
         -5.4398193,   -6.031872 ,   -6.6387405,   -7.262203 ,
         -7.908844 ,   -8.589478 ,   -9.318268 ,  -10.111885 ,
        -10.988968 ,  -11.969719 ,  -13.075821 ,  -14.330429 ,
        -15.758209 ,  -17.385468 ,  -19.240036 ,  -21.351212 ,
        -23.74945  ,  -26.46576  ,  -29.531128 ,  -32.97531  ,
        -36.825653 ,  -41.105377 ,  -45.831757 ,  -51.014282 ,
        -56.652405 ,  -62.733704 ,  -69.232666 ,  -76.10962  ,
        -83.31067  ,  -90.76892  ,  -98.40637  , -106.13696  ,
       -113.87024  , -121.5155   , -128.9867   , -136.20483  ,
       -143.1023   , -149.62476  , -155.73108  , -161.39502  ,
       -166.60352  , -171.35522  , -175.65894  , -179.531    ,
       -182.99463  , -186.07593  , -188.80469  , -191.21118  ,
       -193.3252   , -195.17676  , -196.79395  , -198.20215  ,
       -199.42627  , -200.48877  , -201.40869  , -202.20459  ,
       -202.89209  , -203.48535  , -203.99707  ], dtype=float32)
Dimensions without coordinates: z